# Exercício 2: Sessão síncrona de comunicação segura entre dois agentes usando Curvas Elípticas

Neste segundo exercício, criamos uma versão do esquema do primeiro exercício recorrendo ao uso de Curvas Elípticas. 

Assim, em vez do protocolo de acordo de chaves o Diffie–Hellman substituímos pelo Elliptic-curve Diffie–Hellman e o Digital Signature Algorithm pelo Elliptic Curve Digital Signature Algorithm. Desta forma, foram necessárias poucas alterações à variante que não usa curvas elípticas. Posto isto, as operações em que foram realizadas mais alterações foram as de inicialização das estruturas. Constatamos que a fase de inicialiação do algoritmo é mais rápida no caso das curvas elípticas. 

In [11]:
from Auxs import cypher, decypher, hashs, mac
from BiConn import BiConn
from cryptography.exceptions import InvalidKey, InvalidSignature
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.asymmetric.ec import ECDH
from cryptography.hazmat.primitives.ciphers.algorithms import AES
from cryptography.hazmat.primitives.ciphers import Cipher, modes
from cryptography.hazmat.primitives import hashes, hmac, serialization
from datetime import date
import io, os

password_Emitter = b'1234'
password_Receiver = b'4321'

default_algorithm = hashes.SHA256

In [12]:
class ECDSA(object):
    """
    Armazena o par de chaves ECDSA
    """
    def __init__(self):
        """
        Gera o par de chaves ECDSA
        """
        self.private_key = ec.generate_private_key(
                            ec.SECP384R1,default_backend())

        self.public_key = self.private_key.public_key()

In [13]:
def Gen_Key(priv_file="pk.pem",pub_file="pub.pem", password=None):
    """ 
    Gera as chaves ECDSA publica e privada e armazena-as em ficheiro
    Argumentos:
        pk_file -- string com caminho para ficheiro que armazenará a chave privada
        pub_file -- string com caminho para ficheiro que armazenará a chave publica
        password -- string com a password da chave privada
    """
    ECDSA_object = ECDSA()

    pk_pem = ECDSA_object.private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.BestAvailableEncryption(password))

    pub_pem = ECDSA_object.public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo)

    fd = open(priv_file,"wb"); fd.write(pk_pem); fd.close()
    
    fd = open(pub_file,"wb"); fd.write(pub_pem); fd.close()

#Gera o par de chaves do Emitter
Gen_Key("privadaEmitter.pem","publicaEmitter.pem",password_Emitter)
#Gera o par de chaves do Receiver
Gen_Key("privadaReceiver.pem","publicaReceiver.pem",password_Receiver)

In [14]:
def ecdh(conn, ecdsa_private_key=None, ecdsa_peer_public_key=None):
    """
    Estabelece o protocolo ECDH com autenticação dos agentes através do esquema de assinaturas 
    ECDSA.
    Argumentos: 
        conn -- conexão entre os agentes 
        dsa_private_key -- chave DSA privada para produzir assinatura
        dsa_peer_public_key -- chave DSA publica para verificar a assinatura do outro agente
    Valor de retorno: chave de sessão Diffie-Hellman 
    """
    # Em cada sessão é gerada uma chave publica e privada, enviando-se a primeira ao outro agente
    privKey = ec.generate_private_key(ec.SECP384R1,default_backend())
    pubKey = privKey.public_key().public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo)
    conn.send(pubKey)
    
    # Recebe a chave pública do outro agente e calcula a shared_key
    peer_pub_key_bytes = conn.recv()
    peer_pub_key = serialization.load_pem_public_key(
            peer_pub_key_bytes,
            backend=default_backend())
    
    sk = privKey.exchange(ec.ECDH(),peer_pub_key)
    shared_key = hashs(sk)
    
    hmac_key = mac(shared_key,shared_key)
    
    conn.send(hmac_key)
    hmac_peer = conn.recv()
    
    mac(shared_key,shared_key,hmac_peer)
    
    # Se as chaves ECDSA foram passadas como parâmetro autenticam-se os agentes
    if ecdsa_private_key is not None and ecdsa_peer_public_key is not None:
        # O agente assina a shared_key com a sua chave privada ECDSA, de seguida cifra esta 
        # assinutura com a shared_key da sessão e envia para o outro agente juntamente com o nonce
        signature = ecdsa_private_key.sign(pubKey + peer_pub_key_bytes, ec.ECDSA(hashes.SHA256()))
        nonce, cypher_signature = cypher(shared_key,signature)
        conn.send((nonce,cypher_signature))
        
        # De seguida recebe este par do outro agente de modo a verificar a shared_key
        (peer_nonce, peer_cypher_signature) = conn.recv()
    
        # Primeiro decifra o cyphertext retirando a assinatura enviada pelo outro agente e de 
        # seguida verifica a assinatura usando a chave pública ECDSA do outro agente
        peer_signature = decypher(shared_key,peer_cypher_signature,peer_nonce)
        ecdsa_peer_public_key.verify(peer_signature,peer_pub_key_bytes + pubKey, ec.ECDSA(hashes.SHA256()))
        
    return shared_key

    #Eliminar dados
    privKey = None
    pubKey = None
    peer_pub_key_bytes = None
    peer_pub_key = None
    shared_key = None
    hmac_peer = None

In [15]:
message_size = 2**10

def Emitter(conn):
    """
    Agente que envia uma mensagem.
    """
    # Lê a chave ECDSA pública do Receiver para o autenticar
    with open("publicaReceiver.pem", "rb") as fd:
        ecdsa_peer_public_key = serialization.load_pem_public_key(
            fd.read(),
            backend=default_backend())
    fd.close()
    
    # O BiConn não funciona se ambos os processos utilizarem o stdin
    # password = bytes(getpass.getpass('password '),'utf-8') 
    
    # Lê a sua chave ECDSA privada para se autenticar
    with open("privadaEmitter.pem", "rb") as fd:
        ecdsa_private_key = serialization.load_pem_private_key(
            fd.read(),
            password=password_Emitter,
            backend=default_backend())
    fd.close()
    
    # Estabelece a chave de sessão
    key = ecdh(conn,ecdsa_private_key,ecdsa_peer_public_key)
    
    # Cria um input stream com a mensagem a enviar
    inputs = io.BytesIO(bytes('1'*message_size,'utf-8'))
        
    # nonce para inicialização do modo CRT
    nonce  = os.urandom(16)
    
    # Dados associados
    dadosAssociados = bytes(str(date.today()),'utf-8')
    
    # geração da chave e do contexto de cifra
    cipher = Cipher(AES(key), modes.CTR(nonce), 
                        backend=default_backend()).encryptor()

    # Gerar um mac e inicializa-lo com os dados associados
    mac = hmac.HMAC(key,default_algorithm(),default_backend())
    mac.update(dadosAssociados)

    # comunicação e operação de cifra    
    conn.send((nonce,mac.copy().finalize(),dadosAssociados))
    # define um buffer para onde vão ser lidos, sucessivamente, os vários blocos do input
    buffer = bytearray(32) 
    
    # lê, cifra e envia sucessivos blocos do input 
    try:     
        while inputs.readinto(buffer): 
            ciphertext = cipher.update(bytes(buffer))
            mac.update(ciphertext)
            conn.send((ciphertext, mac.copy().finalize()))         
        
        conn.send((cipher.finalize(), mac.finalize()))    # envia a finalização
    except Exception as err:
        print("Erro no emissor: {0}".format(err))

    inputs.close()          # fecha a 'input stream'
    conn.close()            # fecha a conecção
    
    #Eliminar dados
    key = None 

In [16]:
def Receiver(conn):
    """
    Agente que recebe a mensagem
    """
    # Lê a chave ECDSA pública do Emitter para autenticar o outro agente 
    with open("publicaEmitter.pem", "rb") as fd:
        ecdsa_peer_public_key = serialization.load_pem_public_key(
            fd.read(),
            backend=default_backend())
    fd.close()
    
    # O BiConn não funciona se ambos os processos utilizarem o stdin
    #password = bytes(getpass.getpass('password '),'utf-8')
    
    # Lê a sua chave ECDSA privada para se autenticar
    with open("privadaReceiver.pem", "rb") as fd:
        ecdsa_private_key = serialization.load_pem_private_key(
            fd.read(),
            password=password_Receiver,
            backend=default_backend())
    fd.close()

    # Estabelece a chave de sessão
    key = ecdh(conn,ecdsa_private_key,ecdsa_peer_public_key)

    # Inicializa um output stream para receber o texto decifrado
    outputs = io.BytesIO()

    # Recuperar a informação de nonce e salt
    nonce,tag1,rec = conn.recv()
    
    # geração da chave e do contexto de cifra
    cipher = Cipher(AES(key), modes.CTR(nonce), 
                            backend=default_backend()).encryptor()
    
    # Gera um mac e inicializa-lo com os dados associados
    mac = hmac.HMAC(key,default_algorithm(),default_backend())
    mac.update(rec)

    # Verifica se os dados associados não foram corrompidos
    mac.copy().verify(tag1)

    # operar a cifra: ler da conecção um bloco, autenticá-lo, decifrá-lo e escrever o 
    # resultado no 'stream' de output
    try:
        while True:    
            try:
                ciphertext, tag = conn.recv()
                mac.update(ciphertext)
                if tag != mac.copy().finalize():
                    raise InvalidSignature("erro no bloco intermédio")
                if len(ciphertext) == 0:
                    raise EOFError
                outputs.write(cipher.update(ciphertext))


            except EOFError:
                if tag != mac.finalize():
                    raise InvalidSignature("erro na finalização")                
                outputs.write(cipher.finalize())
                break
            except InvalidSignature as err:
                raise Exception("autenticação do ciphertext ou metadados: {}".format(err))
        print(outputs.getvalue())     # verificar o resultado

    except Exception as err:
        print("Erro no receptor: {0}".format(err))

    outputs.close()    # fechar 'stream' de output
    conn.close()       # fechar a conecção
    #Eliminar dados
    key = None

In [17]:
BiConn(Emitter, Receiver).auto()

b'11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111